In [18]:
# Cell 1: Setup and Verification
import torch
from ultralytics import YOLO
import os

print("✅ Environment Check:")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only'}")
print(f"Ultralytics: {YOLO('yolov8n.pt').info}")

# Verify dataset structure
print("\n📁 Dataset Structure:")
print("Contents:", os.listdir('.'))
print("Train exists:", os.path.exists('train'))
print("Valid exists:", os.path.exists('valid'))
print("Test exists:", os.path.exists('test'))

✅ Environment Check:
PyTorch: 2.7.1+cu118
CUDA: True
GPU: NVIDIA GeForce GTX 1060 with Max-Q Design
Ultralytics: <bound method Model.info of YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(

In [19]:
# Cell 2: Quick Dataset Check
import yaml

with open('data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)

print("📊 Dataset Configuration:")
print(f"Classes: {data_cfg['nc']}")
print(f"Class Names: {data_cfg['names']}")
print(f"Train path: {data_cfg['train']}")
print(f"Val path: {data_cfg['val']}")

📊 Dataset Configuration:
Classes: 4
Class Names: ['0', 'Hand Granade', 'Land Mine', '3']
Train path: ../train/images
Val path: ../valid/images


In [ ]:
# Cell 3: Training (Single Cell)
from ultralytics import YOLO

# Load model and train
model = YOLO('yolov8n.pt')
# print(model.info)
results = model.train(
    data='data.yaml',
    epochs=64,
    imgsz=640,
    batch=8,
    device=0,
    name='yolov8n_grenade_mines',
    patience=15,
    save=True
)

print("🎉 Training completed!")

Ultralytics 8.3.209 🚀 Python-3.12.3 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1060 with Max-Q Design, 6066MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=64, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_grenade_mines, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=

In [ ]:
# Cell 4: Quick Validation
model = YOLO('runs/detect/yolov8n_grenade_mines/weights/best.pt') #/home/turu/Documents/Automech/AEDDRA/RoboFlow/Land-Mines-or-Hand-Grenade.v1i.yolov8/runs/detect/yolov8n_grenade_mines4/weights
metrics = model.val()
print(f"📈 mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# Cell 5: Export for Raspberry Pi
model.export(format='onnx')
print("✅ Model exported to ONNX format")

In [ ]:
import glob
import random
import matplotlib.pyplot as plt
import os

# Make predictions
results = model.predict('test/images/', save=True, conf=0.5)
print(f"🔍 Predictions saved in: {results[0].save_dir}")

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# Quantize model from FP32 to INT8
def quantize_model():
    input_model = 'runs/detect/yolov8n_grenade_mines/weights/best.onnx'
    output_model = 'runs/detect/yolov8n_grenade_mines/weights/best_qnt.onnx'
    
    quantize_dynamic(
        input_model,
        output_model,
        weight_type=QuantType.QUInt8
    )
    
    print(f"✅ Model quantized: {output_model}")
    print("📊 Size reduced for faster Raspberry Pi inference")

In [ ]:
if __name__ == "__main__":
    quantize_model()

ValidationError: Unable to parse proto from file: best.onnx. Please check if it is a valid protobuf file of proto. 